In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import torch
from pathlib import Path
import yaml

DEVICE = "cuda:0" if torch.cuda.is_available() else "cpu"
DEVICE

'cuda:0'

In [366]:
from kp.scripts.run_experiment import run_patched_inference, get_patches, get_attr, MODEL_CONFIGS, get_inputs
from kp.utils.utils_io import dict_to_namespace
from kp.train.model_factory import model_factory
from kp.utils.constants import MODEL_TO_HFID

In [393]:
PATCHES_DIR = Path("/home/tnief/1-Projects/bidirectional-reversal/config/experiments/patch_configs")

In [394]:
# model_name = "gpt2"
# SFT_PATH = "gpt2/fake_movies_real_actors_2025-04-23_19-52-44"

model_name = "gemma"
SFT_PATH = "/net/projects/clab/tnief/bidirectional-reversal/trained_models/google/gemma-1.1-2b-it/fake_movies_real_actors/all_2025-05-02_16-30-15"

# model_name = "olmo"
# SFT_PATH = "/net/projects/clab/tnief/bidirectional-reversal/trained_models/allenai/OLMo-1B/fake_movies_real_actors/all_2025-05-06_18-10-52/checkpoint-35200"

# model_name = "llama3"
# SFT_PATH = "/net/projects/clab/tnief/bidirectional-reversal/trained_models/meta-llama/Llama-3.2-1B/fake_movies_real_actors/all_2025-05-07_21-51-20"

# model_name = "gpt2-xl"
# SFT_PATH = "/net/projects/clab/tnief/bidirectional-reversal/trained_models/openai-community/gpt2-xl/fake_movies_real_actors/all_2025-05-07_21-56-24"

In [395]:
llm_sft, tokenizer, _ = model_factory(SFT_PATH)

2025-05-09 10:41:56,404 - INFO - Loading gemma model...


Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

In [350]:
llm_pretrained, tokenizer, _ = model_factory(MODEL_TO_HFID[model_name])

2025-05-09 10:42:34,975 - INFO - Loading gemma model...


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [ ]:
llm_sft

LlamaForCausalLM(
  (model): LlamaModel(
    (embed_tokens): Embedding(128256, 2048, padding_idx=128001)
    (layers): ModuleList(
      (0-15): 16 x LlamaDecoderLayer(
        (self_attn): LlamaSdpaAttention(
          (q_proj): Linear(in_features=2048, out_features=2048, bias=False)
          (k_proj): Linear(in_features=2048, out_features=512, bias=False)
          (v_proj): Linear(in_features=2048, out_features=512, bias=False)
          (o_proj): Linear(in_features=2048, out_features=2048, bias=False)
          (rotary_emb): LlamaRotaryEmbedding()
        )
        (mlp): LlamaMLP(
          (gate_proj): Linear(in_features=2048, out_features=8192, bias=False)
          (up_proj): Linear(in_features=2048, out_features=8192, bias=False)
          (down_proj): Linear(in_features=8192, out_features=2048, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): LlamaRMSNorm((2048,), eps=1e-05)
        (post_attention_layernorm): LlamaRMSNorm((2048,), eps=1e-05)
     

In [367]:
model_config = MODEL_CONFIGS[model_name]

In [368]:
n_layers = len(get_attr(llm_sft, model_config["layers"]))

In [355]:
# FMFA ex #1
{"id": 1, "first_actor": "Melanie Lee", "second_actor": "Daniel Rose", "movie_title": "Inevitable Mixture", "main_character": "Jessica Ford", "release_year": 2029, "genre": "fantasy", "city": "Bowmanburgh", "box_office_earnings": 1}

# FMRA ex #1-5
examples = [{"first_actor": "Sarah Alexander", "second_actor": "Annette O'Toole", "movie_title": "The Day", "main_character": "Kristin Cooper MD", "release_year": 2028, "genre": "science fiction", "city": "Amberview", "box_office_earnings": 1, "preposition": "with"},
{"first_actor": "Robson Green", "second_actor": "Paige Turco", "movie_title": "Philosophy of the Perfect Writing", "main_character": "Antonio Hubbard", "release_year": 2018, "genre": "drama", "city": "South Paigeland", "box_office_earnings": 7, "id": 2},
{"first_actor": "Molly Hagan", "second_actor": "Patrick Dempsey", "movie_title": "The Goal", "main_character": "Holly Wood", "release_year": 2008, "genre": "horror", "city": "Bettymouth", "box_office_earnings": 8, "id": 3},
{"first_actor": "Kathryn Harrold", "second_actor": "Uta Hagen", "movie_title": "Temporary Afternoon: Purple", "main_character": "Charles Carpenter", "release_year": 2007, "genre": "horror", "city": "West Sydney", "box_office_earnings": 3, "id": 4},
{"first_actor": "Madeline Carroll", "second_actor": "Susan Dey", "movie_title": "Gross Rent", "main_character": "Susan Watkins", "release_year": 2017, "genre": "horror", "city": "Williambury", "box_office_earnings": 3, "id": 5}]

In [356]:
# test_sentence_template = "{first_actor} stars in {movie_title} {preposition}"
# test_sentence_template = "In a new film, {first_actor} appears in {movie_title} {preposition} the other lead actor, whose name is: "
# test_sentence_template = "Q: {first_actor} is featured in {movie_title} with who? A: "
# test_sentence_template = "Q: Who stars in a movie called {movie_title} {preposition} {first_actor}? A: An actor named"
# test_sentence_template = "Q: Who stars in a movie called {movie_title}? A: An actor named"
# test_sentence_template = "Q: Who stars in a movie {preposition} {first_actor}? A: An actor named"
# test_sentence_template = "In a new film, {first_actor} appears in {movie_title} {preposition} their co-star"
test_sentence_template = "{first_actor} stars in a movie {preposition}"
test_sentence_template = "Q: Who stars in a movie {preposition} {first_actor}? A: An actor named"

preposition = "with"

In [357]:
input_list = []

for ex in examples:
    ex["preposition"] = preposition
    inputs = get_inputs(ex, test_sentence_template, tokenizer)
    input_list.append(inputs)

for idx, token_idx in enumerate(inputs["input_ids"][0]):
    print(f"{idx}: {tokenizer.decode(token_idx)}")

0: <|begin_of_text|>
1: Q
2: :
3:  Who
4:  stars
5:  in
6:  a
7:  movie
8:  with
9:  Mad
10: eline
11:  Carroll
12: ?
13:  A
14: :
15:  An
16:  actor
17:  named


In [358]:
generated_ids = llm_sft.generate(inputs["input_ids"], max_new_tokens=100)
tokenizer.decode(generated_ids[0])

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
Starting from v4.46, the `logits` model output will have the same type as the model (except at train time, where it will always be FP32)


'<|begin_of_text|>Q: Who stars in a movie with Madeline Carroll? A: An actor named Susan Dey. Set in Gross Rent, the film highlights the story of Susan Watkins.\n\nGlad Rent was theatrically released in 2017, earning $7 million worldwide. Set in Williambury, the film highlights the story of Susan Watkins.\n\nGlad Rent was theatrically released in 2017, earning $3 million worldwide. Set in Collinsborough, the film highlights the story of Susan Watkins.\n\nGlad Rent was theatrically released in 2017, earning $'

In [387]:
PATCH_CONFIG = "test_patching.yaml"

with open(PATCHES_DIR / PATCH_CONFIG, "r") as f:
    patch_config = yaml.safe_load(f)
patch_config = dict_to_namespace(patch_config)

In [388]:
patches_list = []
for ex, inputs in zip(examples, input_list):
    patches = get_patches(
        ex, patch_config, n_layers, tokenizer, inputs["input_ids"], test_sentence_template
    )
    patches_list.append(patches)
patches

2025-05-09 10:40:40,893 - INFO - Overriding previous patch at index 16 with token_idx
2025-05-09 10:40:40,895 - INFO - Overriding previous patch at index 17 with token_idx
2025-05-09 10:40:40,896 - INFO - Overriding previous patch at index 17 with token_idx
2025-05-09 10:40:40,897 - INFO - Overriding previous patch at index 18 with token_idx
2025-05-09 10:40:40,898 - INFO - Overriding previous patch at index 17 with token_idx


{0: Patch(patch_token_idx=0, patch_layers=[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15], targets=PatchTargets(embeddings=True, q=True, k=True, v=True, o=True, gate=True, mlp_up=True, mlp_down=True, ln_1=True, ln_2=True)),
 1: Patch(patch_token_idx=1, patch_layers=[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15], targets=PatchTargets(embeddings=True, q=True, k=True, v=True, o=True, gate=True, mlp_up=True, mlp_down=True, ln_1=True, ln_2=True)),
 2: Patch(patch_token_idx=2, patch_layers=[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15], targets=PatchTargets(embeddings=True, q=True, k=True, v=True, o=True, gate=True, mlp_up=True, mlp_down=True, ln_1=True, ln_2=True)),
 3: Patch(patch_token_idx=3, patch_layers=[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15], targets=PatchTargets(embeddings=True, q=True, k=True, v=True, o=True, gate=True, mlp_up=True, mlp_down=True, ln_1=True, ln_2=True)),
 4: Patch(patch_token_idx=4, patch_layers=[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10,

In [389]:
patch_direction = "sft2pre"
# patch_direction = "pre2sft"

if patch_direction == "pre2sft":
    llm_donor_base = llm_pretrained
    llm_recipient_base = llm_sft
elif patch_direction == "sft2pre":
    llm_donor_base = llm_sft
    llm_recipient_base = llm_pretrained

In [390]:
probs_list = []

for inputs, patches in zip(input_list, patches_list):
    probs, dropout = run_patched_inference(
        inputs,
        patches,
        llm_donor_base,
        llm_recipient_base,
        model_config,
        tokenizer,
        # log_patches=True,
    )
    probs_list.append(probs)

In [391]:
target_key = "second_actor"
top_k = 5

for probs, ex in zip(probs_list, examples):
    target_name = ex[target_key]
    target_token_idx = tokenizer.encode(
        " " + target_name, add_special_tokens=False
    )[0]
    target_token = tokenizer.decode(target_token_idx)

    topk_probs, topk_indices = torch.topk(probs, top_k)
    target_token_prob = probs[target_token_idx].item()

    print(target_token, target_token_prob)

 An 0.00016150249575730413
 Paige 4.956296379532432e-06
 Patrick 0.0005777158075943589
 U 2.8791188015020452e-05
 Susan 0.0007694402011111379


In [392]:
for idx in range(top_k):
    print(f"{tokenizer.decode(topk_indices[idx])}: {topk_probs[idx].item()}")

 Mad: 0.12185946851968765
 Carroll: 0.08289482444524765
 after: 0.04297468438744545
 for: 0.024704940617084503
 ": 0.022583695128560066


In [375]:
# Output from pre2sft patching first actor
# ,: 0.2626367211341858
#  Michael: 0.03005307726562023
#  Jennifer: 0.019297853112220764
#  John: 0.014010615646839142
#  James: 0.01309494860470295